# Miniproyecto #3: Redes neuronales artificiales (ANN)


**Autores:**
*   Jorge Sebastián Arroyo Estrada CC. 1193482707
*   César Augusto Montoya Ocampo CC. 1036681523

**Tratamiento de Señales III**

**Facultad de Ingeniería**

**Universidad de Antioquia**

---

## Librerías

Importa diversas bibliotecas para el análisis y procesamiento de datos, así como para la implementación y evaluación de modelos de aprendizaje automático y redes neuronales. Se utilizan bibliotecas estándar como `json`, `os`, `pickle` y `datetime` para manejo de archivos y tiempos, además de `itertools` para generar combinaciones de parámetros. `matplotlib`, `numpy` y `pandas` se emplean para visualización y manipulación de datos. Las métricas y herramientas de `sklearn`, como `accuracy_score`, `f1_score`, y `confusion_matrix`, se usan para evaluar el rendimiento de modelos, mientras que `GridSearchCV` y `train_test_split` facilitan la optimización y la partición de los datos. Además, se importan componentes de `keras` como `Sequential`, `Input`, y `Dense` para construir redes neuronales, así como el envoltorio `KerasClassifier` para integrar modelos de `Keras` con `scikit-learn`.

In [4]:
import json
import pickle
from itertools import product
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from classification import main_pipeline, plot_grouped_metrics
from keras import Input
from keras.layers import Dense
from keras.models import Sequential
from scikeras.wrappers import KerasClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    make_scorer,
    precision_score,
    recall_score,
)
from sklearn.multiclass import OneVsRestClassifier

## Globales

Definir rutas para archivos de bases de datos, configurando parámetros globales para un modelo de ML. Establece la ruta base de las características (`DB_BASE_PATH`) y genera listas de nombres y rutas de archivos para cuatro bases de datos con resoluciones de 64x64 y 128x128, diferenciando entre características y PCA. También define la ruta para un archivo de etiquetas (`LABELS_PATH`) y crea un directorio para almacenar los resultados del modelo (`RESULTS_FOLDER`). Se configuran parámetros globales como el estado aleatorio (`RANDOM_STATE`), la validación cruzada (`K`), el estimador con el parámetro de búsqueda de la cuadrícula (`PARAM_GRID`), y las métricas de evaluación como `precision`, `accuracy`, `recall` y `f1_score`. La métrica principal para la optimización del modelo es la puntuación F1 ponderada (`f1_weighted`).

In [5]:
# Database files paths
DB_BASE_PATH = Path("../02_features/")
RESOLUTIONS = 2 * [64, 128]
DB_NAMES = [
    f"DB {RESOLUTIONS[i]}×{RESOLUTIONS[i]}{' PCA' if i >= 2 else ''}" for i in range(4)
]
DB_PATHS = [
    DB_BASE_PATH / f"{'features' if i < 2 else 'pca'}_{RESOLUTIONS[i]}.csv"
    for i in range(4)
]
DB_DICT = {name: path for name, path in zip(DB_NAMES, DB_PATHS)}

# Classes to labels mapping file path
LABELS_PATH = DB_BASE_PATH / "labels.csv"

# Results folder
RESULTS_FOLDER = Path("./ann_results/")
RESULTS_FOLDER.mkdir(parents=True, exist_ok=True)

Define parámetros globales y configura un entorno para crear y evaluar modelos de redes neuronales utilizando `keras` y `scikit-learn`. `RANDOM_STATE` asegura reproducibilidad, mientras que `K` define el número de particiones en validaciones cruzadas. La función `create_model` genera dinámicamente un modelo `Sequential` de `keras`, permitiendo personalizar capas ocultas, funciones de activación, optimizador y métricas. Este modelo se envuelve con `KerasClassifier` para integrarlo con `scikit-learn`.

Se establecen `NEURONS` como una lista de potencias de 2 para definir el número de neuronas, mientras que `ACTIVATIONS` genera combinaciones de funciones de activación (`relu`, `leaky_relu`, `tanh`). `PARAM_GRID` configura un espacio de hiperparámetros para `GridSearchCV`, incluyendo configuraciones dinámicas como dimensiones de entrada y salida del modelo (`input_dim` y `output_dim`).  

Finalmente, se definen métricas de evaluación en `SCORE_METRICS`, incluyendo `accuracy`, `precision`, `recall` y `f1`, ponderadas por clases y con manejo de divisiones por cero. La métrica principal se establece como la última de esta lista (`f1_weighted`).

In [6]:
# Global parameters
RANDOM_STATE = 3
K = 5


# Function to create model, required for KerasClassifier
def create_model(
    neurons,
    activations,
    input_dim,
    output_dim,
    loss="binary_crossentropy",
    optimizer="adam",
):
    """
    Dynamically creates a Keras Sequential model.

    Parameters:
        hidden_layers (int): Number of hidden layers.
        neurons (int): Number of neurons in each hidden layer.
        activation (str): Activation function for the layers.
        optimizer (str): Optimizer for compiling the model.

    Returns:
        model: A compiled Keras model.
    """
    model = Sequential()

    # Input layer
    model.add(Input(shape=(input_dim,)))

    # Hidden layers
    for i, activation in enumerate(activations):
        model.add(Dense(neurons // (2**i), activation=activation))

    # Output layer
    model.add(Dense(units=output_dim, activation="softmax"))

    # Compile model
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )

    return model


ESTIMATOR = KerasClassifier(model=create_model, verbose=0)

NEURONS = [2**i for i in range(4, 9)]
ACTIVATION_FNS = ["relu", "leaky_relu", "tanh"]
ACTIVATIONS = [
    list(combo) for n in range(1, 4) for combo in product(ACTIVATION_FNS, repeat=n)
]

PARAM_GRID = {
    "model__neurons": NEURONS,
    "model__activations": ACTIVATIONS,
    "model__input_dim": None,
    "model__output_dim": None,
    "model__optimizer": ["adam"],
    "model__loss": ["sparse_categorical_crossentropy"],
}

SCORE_METRICS = {
    "accuracy": make_scorer(accuracy_score),
    "precision_weighted": make_scorer(
        precision_score, average="weighted", zero_division=np.nan
    ),
    "recall_weighted": make_scorer(
        recall_score, average="weighted", zero_division=np.nan
    ),
    "f1_weighted": make_scorer(f1_score, average="weighted", zero_division=np.nan),
}
MAIN_SCORE_METRIC = list(SCORE_METRICS.keys())[-1]

## ANN

Este código define varias funciones para manejar el procesamiento de datos, entrenamiento de modelos y almacenamiento de resultados:

- `load_and_preprocess_data`: Carga un archivo CSV, separa las características y las etiquetas, y normaliza las características utilizando `StandardScaler`.
- `load_label_mapping`: Carga un archivo CSV con etiquetas de clases y devuelve un diccionario que mapea los números de clase a nombres legibles.
- `load_results`: Carga los resultados previamente guardados de un experimento, incluyendo el mejor modelo entrenado, métricas de entrenamiento, validación y prueba, y el tiempo de inferencia. Retorna `None` si no se encuentran los archivos o si ocurre un error.
- `save_results`: Guarda los resultados de un experimento en disco, incluyendo las métricas de evaluación, el tiempo de inferencia y el mejor modelo entrenado en formatos JSON y pickle.
- `grid_search_cv`: Realiza una búsqueda en cuadrícula para optimizar los hiperparámetros de un modelo de aprendizaje automático, utilizando validación cruzada y varias métricas de evaluación. Retorna el mejor modelo, las métricas de entrenamiento y validación promedio, y el tiempo de inferencia.
- `evaluate_model`: Evalúa el rendimiento de un modelo calculando métricas como precisión, recall, F1 y exactitud sobre las etiquetas verdaderas y predichas.

Funciones misceláneas para mostrar los resultados en pantalla de manera ordenada:
- `print_results`: Muestra de forma estructurada las métricas de evaluación del modelo para los conjuntos de entrenamiento, validación y prueba, formateándolas como porcentajes. También imprime el tiempo total de inferencia.
- `plot_metrics`: Genera un gráfico de barras que representa las métricas de evaluación del modelo en porcentajes, con el objetivo de visualizar el desempeño de manera clara y comprensible.
- `plot_confusion_matrix`: Crea y visualiza una matriz de confusión basada en las etiquetas reales y predichas, permitiendo identificar patrones de errores en las predicciones del modelo.

El código principal que carga y preprocesa los datos desde un archivo CSV, estandariza las características y obtiene un mapeo de etiquetas (1). Luego divide los datos en conjuntos de entrenamiento y prueba, reservando un 20% para este último (2). Intenta cargar resultados previos para evitar cálculos redundantes (3). Si no existen, realiza una búsqueda de hiperparámetros mediante validación cruzada para encontrar el mejor modelo (4). Posteriormente, evalúa el modelo en el conjunto de prueba, calcula métricas clave, imprime los resultados e informa sobre el tiempo de inferencia (5). Si los resultados son nuevos, los guarda y genera visualizaciones como gráficos de métricas y matrices de confusión (6). Finalmente, retorna las métricas calculadas para entrenamiento, validación y prueba (7).

## Aplicar ANN

Ejecución de todo el código para cada una de las bases de datos disponibles, para ambas resoluciones y cuando se empleó PCA o no para la reducción de dimensionalidad

In [ ]:
test_scores = {}
for name, path in DB_DICT.items():
    _, _, test_score = main_pipeline(
        name,
        path,
        LABELS_PATH,
        RESULTS_FOLDER,
        RANDOM_STATE,
        ESTIMATOR,
        PARAM_GRID,
        K,
        SCORE_METRICS,
        MAIN_SCORE_METRIC,
    )
    test_scores[name] = test_score

Graficar todas las métricas de prueba para cada una de las bases de datos, para así poder ver gráficamente cuál de las bases de datos nos entregó un mejor resultado.

In [ ]:
plot_grouped_metrics(test_scores)